In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torchaudio

In [ ]:
class DeepFeatures2(nn.Module):
    def __init__(self):
        super(DeepFeatures2,self).__init__()
        self.conv1 = nn.Sequential(
                    nn.Conv1d(in_channels=40,out_channels=64,kernel_size=3,stride=1),
                    nn.BatchNorm1d(64),
                    nn.ReLU(),
                    nn.Dropout1d(0.2),
                    nn.Conv1d(in_channels=64,out_channels=64,kernel_size=3,stride=1),
                    nn.BatchNorm1d(64),
                    nn.ReLU(),
                    nn.MaxPool1d(kernel_size=3),
                    nn.Dropout1d(0.2)
                    )
        self.conv2 = nn.Sequential(
                    nn.Conv1d(in_channels=64,out_channels=128,kernel_size=3,stride=1),
                    nn.BatchNorm1d(128),
                    nn.ReLU(),
                    nn.Conv1d(in_channels=128,out_channels=64,kernel_size=3,stride=1),
                    nn.BatchNorm1d(64),
                    nn.ReLU(),
                    nn.MaxPool1d(kernel_size=3)
                    )
        
        #encoder_layers = TransformerEncoderLayer(d_model=64, nhead=4, dim_feedforward=200,batch_first=True)
        #self.encoder = TransformerEncoder(encoder_layers, 2)
        self.attn1 = nn.MultiheadAttention(embed_dim=64, num_heads=4, batch_first=True)
        self.ln1 = nn.LayerNorm([31,64])
        self.mlp1 = nn.Linear(64,64)
        self.ln_mlp1 = nn.LayerNorm([31,64])
        
        self.attn2 = nn.MultiheadAttention(embed_dim=64, num_heads=4, batch_first=True)
        self.ln2 = nn.LayerNorm([31,64])
        self.mlp2 = nn.Linear(64,64)
        self.ln_mlp2 = nn.LayerNorm([31,64])
        
        self.linear = nn.Sequential(
                    nn.Linear(31*128,512),
                    nn.ReLU(),
                    nn.Dropout(0.3),
                    nn.Linear(512,5)
                    )
    
    def forward(self,x):
        x = x.squeeze(1)
        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        
        
        #print(x.shape)
        x = x.permute(0,2,1) # N,31,64 
        z = x.clone()
        
        #print(x.shape)
        y,_ = self.attn1(x,x,x)
        x= y.clamp(min=0) + x
        x = self.ln1(x) # N,31,64
        
        x=self.mlp1(x).clamp(min=0) + x
        x=self.ln_mlp1(x)
        
        y,_ = self.attn2(x,x,x)
        x= y.clamp(min=0) + x
        x = self.ln_mlp2(x) # N,31,64
        
        x=self.mlp2(x).clamp(min=0) + x
        x=self.ln2(x)
        
        x = torch.cat((z,x),dim=2)
        #print(x.shape)
        x = x.view(-1,31*128)
        #print(x.shape)
        x = self.linear(x)
        #print(x.shape)
        return x